In [1]:
#!connect signalr --kernel-name rover --hub-url http://10.0.0.252:1024/kernelhub

Kernel added: #!rover

In [1]:
#!rover
resourceScanner.CaptureFromCamera(roverBody.Camera);
var sign = 1;

In [1]:
#!rover
using PiTop.Interactive.Rover.ImageProcessing;
using System.Threading;
using System.IO;
//resourceScanner.UseModel(new DirectoryInfo("/home/pi/dotnet_conf_resources/models/v5"));
resourceScanner.UseUri(new Uri("https://lobe-diego.ngrok.io/predict/34bc1074-f2fd-48bd-b473-7d1c4b50cd34"));

In [1]:
#!rover
var frame = roverBody.Camera.GetFrame().Preview();
var lastScanResult = resourceScanner.AnalyseFrame(frame);

var GoToQuadrant = new Action<int, IPanTiltController>( (quadrantId, tiltController) => {
    switch(quadrantId){
        case 0:
        tiltController.Pan = Angle.FromDegrees(60);
        tiltController.Tilt = Angle.FromDegrees(30);
        break;        
        case 1:
        tiltController.Pan = Angle.FromDegrees(-75);
        tiltController.Tilt = Angle.FromDegrees(35);
        break;
        case 2:
        tiltController.Pan = Angle.FromDegrees(-50);
        tiltController.Tilt = Angle.FromDegrees(-10);
        break;        
        case 3:
        tiltController.Pan = Angle.FromDegrees(25);
        tiltController.Tilt = Angle.FromDegrees(-20);
        break;
    }
});

In [1]:
#!rover
lastScanResult = null;
roverBody.AllLightsOff();

roverBrain.Perceive = (rover, now, token) => {

};

roverBrain.Plan = (rover, now, token) => {
    Task.Delay(2000).Wait();
    return PlanningResult.NoPlan;
};

roverBrain.Act = (rover, now, token) => { 
    
};

In [1]:
#!rover
//water
GoToQuadrant(0, roverBody.TiltController);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame().Focus();
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(frame);
display(lastScanResult?.Classification);

Label,Confidence
water,1


In [1]:
#!rover
//lightning
GoToQuadrant(1, roverBody.TiltController);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame().Focus();
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(frame);
display(lastScanResult?.Classification);

Label,Confidence
grass,1


In [1]:
#!rover
//metal
GoToQuadrant(2, roverBody.TiltController);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame().Focus();
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(frame);
display(lastScanResult?.Classification);

Label,Confidence
metal,1


In [1]:
#!rover
//grass
GoToQuadrant(3, roverBody.TiltController);
await Task.Delay(1000);
frame = roverBody.Camera.GetFrame().Focus();
lastScanResult = resourceScanner.AnalyseFrame(frame);
display(frame);
display(lastScanResult?.Classification);


Label,Confidence
fighting,1


In [1]:
#!rover
display(frame);
display(lastScanResult.Classification);
display(lastScanResult.Classifications);

In [1]:
#!rover
roverBody.TiltController.Pan = Angle.FromDegrees(0);
roverBody.TiltController.Tilt = Angle.FromDegrees(0);